# Free surface model generator

**Library imports**

In [1]:
import uuid

import numpy as np

from DataModelDict import DataModelDict as DM

import atomman as am

from pathlib import Path

import iprPy

In [2]:
def vectstr(vect):
    out = ''
    for v in vect:
        out += f'{repr(v)} '
    return out.strip()

def gen_stacking_fault_model(hkl, uvws, family, atomshift, shiftvector1, shiftvector2, setting=None):
    
    if len(hkl) == 3:
        model_id = family + '--' + '%i%i%isf' % tuple(hkl)
    elif len(hkl) == 4:
        model_id = family + '--' + '%i%i%i%isf' % (np.abs(hkl[0]), np.abs(hkl[1]),np.abs(hkl[2]),np.abs(hkl[3]))
    if setting is not None:
        model_id += '-' + setting
    fname = Path(model_id + '.json')
    
    if fname.is_file():
        #return None
        old = DM(fname.as_posix())
        key = old['stacking-fault']['key']
    else:
        key = str(uuid.uuid4())
    
    model = DM()
    model['stacking-fault'] = DM()
    model['stacking-fault']['key'] = key
    model['stacking-fault']['id'] = model_id
    model['stacking-fault']['system-family'] = family
    
    model['stacking-fault']['calculation-parameter'] = DM()
    if len(hkl) == 3:
        model['stacking-fault']['calculation-parameter']['a_uvw'] = '%2i %2i %2i' % tuple(uvws[0])
        model['stacking-fault']['calculation-parameter']['b_uvw'] = '%2i %2i %2i' % tuple(uvws[1])
        model['stacking-fault']['calculation-parameter']['c_uvw'] = '%2i %2i %2i' % tuple(uvws[2])
    elif len(hkl) == 4:
        model['stacking-fault']['calculation-parameter']['a_uvw'] = vectstr(uvws[0])
        model['stacking-fault']['calculation-parameter']['b_uvw'] = vectstr(uvws[1])
        model['stacking-fault']['calculation-parameter']['c_uvw'] = vectstr(uvws[2])
    model['stacking-fault']['calculation-parameter']['cutboxvector'] = 'c'
    model['stacking-fault']['calculation-parameter']['atomshift'] = vectstr(atomshift)
    model['stacking-fault']['calculation-parameter']['faultpos'] = '0.0'  
    model['stacking-fault']['calculation-parameter']['shiftvector1'] = vectstr(shiftvector1)
    model['stacking-fault']['calculation-parameter']['shiftvector2'] = vectstr(shiftvector2)
    
    with open(model_id + '.json', 'w') as f:
        model.json(fp=f, indent=4)

In [3]:
database = iprPy.load_database('master')
protos = database.get_records_df(style='crystal_prototype')

# Limit to elemental prototypes
protos = protos[protos.natypes == 1]

In [4]:
np.unique(protos.id)

array(['A1--Cu--fcc', 'A15--beta-W', 'A2--W--bcc',
       "A3'--alpha-La--double-hcp", 'A3--Mg--hcp', 'A4--C--dc',
       'A5--beta-Sn', 'A6--In--bct', 'A7--alpha-As', 'Ah--alpha-Po--sc'],
      dtype=object)

## A1--Cu--fcc

In [5]:
family = 'A1--Cu--fcc'
ucell = am.load('system_model', Path(iprPy.libdir, 'crystal_prototype', family+'.json').as_posix())

a = 2 ** -0.5
primbox = am.Box.trigonal(a=a, alpha=60)
conventional_setting = 'f'

hkls = [
    [1,0,0],
    [1,1,1],
       ]

## A2--W--bcc

In [7]:
family = 'A2--W--bcc'
ucell = am.load('system_model', Path(iprPy.libdir, 'crystal_prototype', family+'.json').as_posix())

a = 2 * 3 ** -0.5
alpha = am.tools.vect_angle([1, 1, 1], [1, -1,-1])
primbox = am.Box.trigonal(a=a, alpha=alpha)
conventional_setting = 'i'

hkls = [
    [1,1,0],
    [1,1,2],
    [1,2,3],
       ]

## A3--Mg--hcp

In [9]:
family = 'A3--Mg--hcp'
ucell = am.load('system_model', Path(iprPy.libdir, 'crystal_prototype', family+'.json').as_posix())

primbox = ucell.box
conventional_setting = 'p'

hkls = [
    [0,0,0,1],
    [1,0,-1,0],
    [1,0,-1,1],
    [2,-1,-1,2],
       ]

## Generate

In [10]:
for hkl in hkls:
    
    # Use primitive ucell to find shortest lattice shift vectors
    puvws = am.defect.free_surface_basis(hkl, box=primbox, conventional_setting=conventional_setting)
    
    if len(hkl) == 3:
        pcuvws = am.tools.miller.vector_primitive_to_conventional(puvws, setting=conventional_setting)
    elif len(hkl) == 4:
        pcuvws = puvws
    shiftvector1 = pcuvws[0]
    shiftvector2 = pcuvws[1]
    
    # Use conventional ucell for rotations
    uvws = am.defect.free_surface_basis(hkl, box=ucell.box)
    
    # Rotate ucell
    system = ucell.rotate(uvws)

    # Get unique scaled z coordinates
    allsz = system.atoms_prop(key='pos', scale=True)[:,2]

    sz = np.array([])
    for z in allsz:

        if np.isclose(z, 1.0) or np.isclose(z, 0.0):
            z = 0.0

        if np.sum(np.isclose(sz, z)) == 0:
            sz = np.append(sz, z)

    sz = np.sort(sz)

    # Wraparound
    atomshifts = [np.array([0.0, 0.0, ((sz[-1] - 1 + sz[0]) / 2)])]
    deltas = [sz[0] - (sz[-1] - 1)]

    # All middle
    for i in range(len(sz)-1):
        deltatest = sz[i+1] - sz[i]
        newdelta = True
        for delta in deltas:
            if np.isclose(delta, deltatest):
                newdelta = False
                break
        if newdelta:
            atomshifts.append(np.array([0.0, 0.0, ((sz[i] + sz[i+1]) / 2)]))
            deltas.append(deltatest)

    if len(atomshifts) == 1:
        gen_stacking_fault_model(hkl, uvws, family, atomshifts[0], shiftvector1, shiftvector2)
    else:
        for i, atomshift in enumerate(atomshifts):
            gen_stacking_fault_model(hkl, uvws, family, atomshift, shiftvector1, shiftvector2, setting=str(i+1))

In [33]:
puvws

array([[ 0.66666667, -0.33333333, -0.33333333,  0.        ],
       [ 0.33333333,  0.33333333, -0.66666667,  0.        ],
       [ 0.        ,  0.        , -0.        ,  1.        ]])

In [36]:
uvws[0]

array([-1.,  1., -0.,  0.])

In [37]:
uvws[1]

array([-0.33333333, -0.33333333,  0.66666667,  1.        ])